# TMDLs and DMRs
### Goal: understand the effect of TMDL on DMR values
Author: Ryan Treves

In [49]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
import scipy.stats as ss

# Suppress warning messages
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 2000
# display all rows & columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [50]:
# Load ATTAINS data
AUs = pd.read_csv('all_AUs.csv')
assessments = pd.read_csv('all_NotSupporting_assessments.csv', dtype={'reportingCycleText': str}, parse_dates=['assessment_date'], date_parser=lambda t: pd.to_datetime(t, errors='coerce'))
tmdls = pd.read_csv('all_actions.csv', parse_dates=['completionDate', 'TMDLDate'], date_parser=lambda t: pd.to_datetime(t, errors='coerce')).drop('Unnamed: 0', axis=1).rename(columns={'assessmentUnitIdentifier':'AUID'})

# Load RAD Data
# HUC AUID link table (source: `get_huc_AUID_links.R`)
HUC_AUID_links = pd.read_csv('huc_AUID_links.csv', dtype={'attributes.huc12': object})
HUC_AUID_links = HUC_AUID_links.drop('Unnamed: 0', axis=1).rename(columns={'attributes.huc12': 'HUC-12', 'attributes.assessmentunitidentifier':'AUID'})
# HUC NPDES permit link table (source: `get_huc_npdes_permit_links.R`)
HUC_npdes_permit_links = pd.read_csv('huc_npdes_permit_links.csv', dtype={'attributes.WBD_HUC12': object})
HUC_npdes_permit_links = HUC_npdes_permit_links.drop('Unnamed: 0', axis=1).rename(columns={'attributes.WBD_HUC12': 'HUC-12'})
huc_AUID_links = pd.read_csv('huc_AUID_links.csv', dtype={'attributes.huc12': object}).rename(columns={'attributes.assessmentunitidentifier':'AUID', 'attributes.huc12': 'HUC-12'})

Let's start with just one state: Texas

In [3]:
# Load TX DMR Data
TX_dmrs = pd.read_csv('TX_dmrs.csv', dtype={'dmr_value_standard_units': float,
                              'parameter_code': object},
                       parse_dates=['monitoring_period_end_date'],
                       date_parser=lambda t: pd.to_datetime(t, errors='coerce'))

# Load permit data and subset to just TX, and permits with valid issue dates
permit_data = pd.read_csv('permit_data.csv', parse_dates=['effective_date', 'issue_date', 'original_effective_date', 'original_issue_date', 'expiration_date', 'retirement_date', 'termination_date'], date_parser=lambda t: pd.to_datetime(t, errors='coerce'))
TX_permit_data = permit_data[permit_data['permit_state']=='TX'].dropna(subset='issue_date')
del permit_data

# Merge with permit_data to get permit issuance date
TX_dmrs = TX_dmrs.merge(TX_permit_data, on=['npdes_permit_id', 'version_nmbr'], how='left')
del TX_permit_data

In [4]:
# Subset to TX and delete large tmdl table to save memory
TX_tmdls = tmdls[tmdls['state_code']=='TX']
del tmdls

In [69]:
# Link TMDLs to HUC-12 codes to NPDES permittees
TX_tmdls_merged = TX_tmdls.drop_duplicates().merge(HUC_AUID_links, on='AUID', how='inner').merge(HUC_npdes_permit_links, on='HUC-12', how='inner')
del HUC_AUID_links
del HUC_npdes_permit_links

# Filter down to TMDLs that are classified as point source
TX_tmdls_merged = TX_tmdls_merged[TX_tmdls_merged['pollutantSourceTypeCode'] =='Point source']

Now we need to merge DMRs with TMDLs on NPDES permit ID and pollutant.

In [71]:
# Dictionary linking `pollutantName` from ATTAINS to one or more values of `parameter_desc` from ICIS-NPDES. Note: there is definitely some uncertainty involved in this linkage due to my lack of expertise in water chemistry (e.g., are there point-source parameters not related to BOD that may still contribute to water body impairment caused by BOD?).
# Copied matching parameters from the list of values of `parameter_desc` in DMR Data dictionary, minus those indicating '% removed'. Keys are names of pollutants from the list of all point-source TMDLs.
pollutant_parameter_link_dict = {'ESCHERICHIA COLI (E. COLI)':['E. coli', 'E. coli exceedances', 'E. coli, colony forming units [CFU]', 'E. coli, MTEC-MF', 'E. coli, thermotol, MF, MTEC'],
                             'ENTEROCOCCUS BACTERIA': ['Enterococci Enterococci - % of upper confidence limit', 'Enterococci - % samples exceeding limit', 'Enterococci: group D, MF trans, M-E, EIA'],

                             'FECAL COLIFORM': ['Fecal coliform', 'Fecal coliform in sludge, CFU/gram', 'Fecal coliform, MPN, EC med, 44.5 C'],

                             'SELENIUM': ['Selenium, acid soluble', 'Selenium, dissolved [as Se]', 'Selenium, dry weight', 'Selenium, potentially dissolvd', 'Selenium, sludge solid', 'Selenium, sludge, total, dry weight [as Se]', 'Selenium, total [as Se]', 'Selenium, total recoverable'],

                             'BIOCHEMICAL OXYGEN DEMAND (BOD)':['Non-nitrogenous BOD', 'Carbonaceous BOD, 5 day, 20 deg C filtrd', 'BOD','BOD 5-Day [lbs/1000 lbs Product]','BOD excursions','BOD, [ult. 1st stage]','BOD, [ult. 2nd stage]','BOD, [ult. all stages]','BOD, % over influent','BOD, 11-day [20 deg. C]','BOD, 120 Day, 20 Degrees C','BOD, 20-day [20 deg. C]','BOD, 20-day, percent removal','BOD, 35-day [20 deg. C]','BOD, 5 day carbonaceous, % rmvl, dry','BOD, 5 day carbonaceous, % rmvl, wet','BOD, 5 day percent removal, dry','BOD, 5 day percent removal, wet','BOD, 5 lb/cu ft process','BOD, 5-day','BOD, 5-day, 20 deg C per cfs of streamflw','BOD, 5-day, 20 deg. C','BOD, 5-day, 20 deg. C','BOD, 5-day, 20 deg. C','BOD, 5-day, 20 deg. C [per season]','BOD, 5-day, 20 deg. C per production','BOD, 5-day, dissolved','BOD, 5-day, percent removal','BOD, 5-day, total','BOD, carb-5 day, 20 deg C, percent removal','BOD, carbonaceous [20 day, 20 C]','BOD, carbonaceous [25-day, 20 C]','BOD, carbonaceous [28 day, 20 C]','BOD, carbonaceous [40 day, 20 C]','BOD, carbonaceous [5 day, 20 C]','BOD, carbonaceous [5 day, 20 C][load]','BOD, carbonaceous [5 day, 20 C][per discharge]','BOD, carbonaceous [5 day, 5 C]','BOD, carbonaceous [5-day, 20 C][streamflow]','BOD, carbonaceous, 20-day, % Removal','BOD, carbonaceous, percent removal','BOD, filtered, 5 day, 20 deg C','BOD, mass, times flow prop. multiplier','BOD, nitrogen inhib 5-day [20 deg. C]', 'Avg. of 7-day sum of BOD5 values'],

                             'LEAD':['Lead,Lead 210', 'Lead 210, total', 'Lead 212', 'Lead 214', 'Lead-212 [Radioactivity Concentration]', 'Lead, calculated limit', 'Lead, dissolved [as Pb]', 'Lead, Organic, Total', 'Lead, potentially dissolvd', 'Lead, sludge solid', 'Lead, sludge total', 'Lead, sludge, total, dry weight [as Pb]', 'Lead, total [as Pb]', 'Lead, total [as Pb] [per discharge]', 'Lead, total dry weight [as Pb]', 'Lead, total recoverable', 'Lead, total recoverable', 'Lead, total sludge'],

                             'CARBONACEOUS BOD':['BOD, carbonaceous [20 day, 20 C]','BOD, carbonaceous [25-day, 20 C]','BOD, carbonaceous [28 day, 20 C]','BOD, carbonaceous [40 day, 20 C]','BOD, carbonaceous [5 day, 20 C]','BOD, carbonaceous [5 day, 20 C][load]','BOD, carbonaceous [5 day, 20 C][per discharge]','BOD, carbonaceous [5 day, 5 C]','BOD, carbonaceous [5-day, 20 C][streamflow]', 'Carbonaceous BOD, 5 day, 20 deg C filtrd'],

                             'MERCURY':['Mercury','Mercury [Hg], in barite, dry weight', 'Mercury, dissolved [as Hg]','Mercury, potentially dissolvd','Mercury, sludge, total, dry weight [as Hg]','Mercury, tot in bot deposits [dry wgt]','Mercury, total [as Hg]','Mercury, total [as Hg] [per discharge]','Mercury, total [low level]','Mercury, total low level','Mercury, total recoverable','Mercury, total recoverable'],

                             'NICKEL':['Nickel','Nickel carbonyl','Nickel, dissolved [as Ni]','Nickel, potentially dissolvd','Nickel, sludge solid','Nickel, sludge total','Nickel, sludge, total, dry weight [as Ni]','Nickel, suspended [as Ni]','Nickel, tot in bottom deposits [dry wgt]','Nickel, total [as Ni]','Nickel, total [as Ni] [per discharge]','Nickel, total per batch','Nickel, total recoverable','Nickel, total sludge'],

                             'CYANIDE':['Cyanide','Cyanide [A]', 'Cyanide and thiocyanate - total', 'Cyanide complexed to range of compound', 'Cyanide in bottom deposits [dry wgt]','Cyanide, dissolved std method','Cyanide, filterable, total','Cyanide, free [amen. to chlorination]','Cyanide, free [amenable to chlorine]','Cyanide, free available','Cyanide, free not amenable to chlorin.','Cyanide, free-water plus wastewaters','Cyanide, sludge solid','Cyanide, total [as CN]','Cyanide, total recoverable','Cyanide, weak acid, dissociable'],

                             'CHROMIUM, TOTAL':['Chromium','Chromium hexavalent [as Cr]','Chromium III','Chromium, dissolved [as Cr]','Chromium, hexavalent','Chromium, hexavalent','Chromium, hexavalent [as Cr]','Chromium, hexavalent dissolved [as Cr]','Chromium, hexavalent in bot dep [dry wt]','Chromium, hexavalent potentially disolvd','Chromium, hexavalent tot recoverable','Chromium, hexavalent tot recoverable','Chromium, sludge solid','Chromium, sludge total','Chromium, sludge, total, dry weight [as Cr]','Chromium, suspended [as Cr]','Chromium, total','Chromium, total [as Cr]','Chromium, total [as Cr]','Chromium, total [as Cr] [Can Production]','Chromium, total [as Cr] [per discharge]','Chromium, total dry weight [as Cr]','Chromium, total in bot dep [wet wgt]','Chromium, total recoverable','Chromium, trivalent [as Cr]','Chromium, trivalent dissolved, as Cr','Chromium, trivalent in bottom deposits','Chromium, trivalent total recoverable','Chromium, trivalent, potentially dissolvd'],

                             'CADMIUM':['Cadmium','Cadmium [Cd], in barite, dry weight','Cadmium, dissolved [as Cd]','Cadmium, dry weight','Cadmium, in bottom deposits [dry wgt]','Cadmium, percent removal','Cadmium, potentially dissolvd','Cadmium, sludge solid','Cadmium, sludge total','Cadmium, sludge, total, dry weight [as Cd]','Cadmium, total [as Cd]','Cadmium, total [as Cd] [per discharge]','Cadmium, total recoverable'],

                             'AMMONIA-NITROGEN':['Ammonia nitrogen, total, [as N] 30 day', 'Nitrogen, ammonia [NH3-N]','Nitrogen, ammonia [NH3-N], [sludge]','Nitrogen, ammonia dissolved','Nitrogen, ammonia in bottom deposits','Nitrogen, ammonia per cfs of streamflow','Nitrogen, ammonia total [as N]','Nitrogen, ammonia total [as N] [per discharge]','Nitrogen, ammonia total [as N] [per season]','Nitrogen, ammonia total [as NH4]','Nitrogen, ammonia, percent removal','Nitrogen, ammonia, sludge solid','Nitrogen, ammonia, sludge, tot dry wgt','Nitrogen, ammonia, tot unionized [as N]','Nitrogen, ammonia, total [as NH3]','Nitrogen, AmmoniaTotal'],

                             'CHLORDANE':['Chlordane','Chlordane - Nonachlor, Cis','Chlordane - Nonachlor, Trans','Chlordane [tech mix & metabs], dry wgt', 'Chlordane [tech mix. and metabolites]','Chlordane, .alpha.-, dry weight','Chlordane, .alpha.-, whole water','Chlordane, CA Ocean Plan definition'],

                             'AMMONIA':['Ammonia [as N] + unionized ammonia','Ammonia & ammonium- total','Ammonia nitrogen, total, [as N] 30 day','Ammonia, unionized', 'Nitrogen, ammonia [NH3-N]','Nitrogen, ammonia [NH3-N], [sludge]','Nitrogen, ammonia dissolved','Nitrogen, ammonia in bottom deposits','Nitrogen, ammonia per cfs of streamflow','Nitrogen, ammonia total [as N]','Nitrogen, ammonia total [as N] [per discharge]','Nitrogen, ammonia total [as N] [per season]','Nitrogen, ammonia total [as NH4]','Nitrogen, ammonia, percent removal','Nitrogen, ammonia, sludge solid','Nitrogen, ammonia, sludge, tot dry wgt','Nitrogen, ammonia, tot unionized [as N]','Nitrogen, ammonia, total [as NH3]','Nitrogen, AmmoniaTotal'],

                             'ALUMINUM':['Aluminum','Aluminum [total]','Aluminum chloride, dissolved, water','Aluminum oxide','Aluminum stearate, wat sol in drill fl','Aluminum sulfate','Aluminum, acid soluble','Aluminum, dissolved [as Al]','Aluminum, ion','Aluminum, potentially dissolvd','Aluminum, total','Aluminum, total [as Al]','Aluminum, total [as Al] [Can Production]','Aluminum, total recoverable','Aluminum, total sludge, dry wt.'],

                             # TODO
                             'SEDIMENTATION/SILTATION':[],

                             'COPPER':['Copper','Copper as suspended black oxide','Copper in bottom deposits [dry wgt]','Copper, calculated limit','Copper, dissolved [as Cu]','Copper, dry weight','Copper, percent removal','Copper, potentially dissolved','Copper, sludge solid','Copper, sludge total','Copper, sludge, total, dry weight [as Cu]','Copper, suspended [as Cu]','Copper, total [as Cu]','Copper, total [as Cu] [per discharge]','Copper, total per batch','Copper, total recoverable','Copper, total recoverable'],

                             # TODO
                             'DISSOLVED OXYGEN':[],

                             'NITROGEN, TOTAL':['Nitrogen [as NO3] sludge solid','Nitrogen Kjeldahl, total [TKN]','Nitrogen oxides [as N]','Nitrogen sludge solid','Nitrogen sludge total','Nitrogen-nitrate in water, [pct]','Nitrogen-nitrite in water, [pct]','Nitrogen, ammonia [NH3-N]','Nitrogen, ammonia [NH3-N], [sludge]','Nitrogen, ammonia dissolved','Nitrogen, ammonia in bottom deposits','Nitrogen, ammonia per cfs of streamflow','Nitrogen, ammonia total [as N]','Nitrogen, ammonia total [as N] [per discharge]','Nitrogen, ammonia total [as N] [per season]','Nitrogen, ammonia total [as NH4]','Nitrogen, ammonia, percent removal','Nitrogen, ammonia, sludge solid','Nitrogen, ammonia, sludge, tot dry wgt','Nitrogen, ammonia, tot unionized [as N]','Nitrogen, ammonia, total [as NH3]','Nitrogen, AmmoniaTotal','Nitrogen, Dissolved','Nitrogen, inorganic total','Nitrogen, Kjeldahl','Nitrogen, Kjeldahl, dissolved [as N]','Nitrogen, Kjeldahl, total [as N]','Nitrogen, Kjeldahl, total [as N] [per discharge]','Nitrogen, Kjeldahl, total [TKN] [water]','Nitrogen, Kjeldahl, total [TKN], [sludge]','Nitrogen, Kjeldahl,Total[TKN], insoluble','Nitrogen, KjeldahlTotal','Nitrogen, nitrate [NO3], [sludge]','Nitrogen, nitrate [NO3], [water]','Nitrogen, nitrate dissolved','Nitrogen, nitrate total [as N]','Nitrogen, nitrate total [as NO3]','Nitrogen, NitrateTotal','Nitrogen, nitrite total [as N]','Nitrogen, nitrite total [as NO2]','Nitrogen, NitriteTotal','Nitrogen, organic total [as N]','Nitrogen, organic, dissolved [as N]','Nitrogen, oxidized','Nitrogen, sludge, total, dry weight [as N]','Nitrogen, Total','Nitrogen, total','Nitrogen, total [as N]','Nitrogen, total [as N]','Nitrogen, total [as N] [adjusted]','Nitrogen, total [as N] [credit]','Nitrogen, total [as N] [delivered]','Nitrogen, Total As N','Nitrogen, Total as NO3 + NH3','Nitrogen, total available [water]','Nitrogen, total available, [sludge]','Nitrogen, total Kjeldahl','Nitrogen, total Kjeldahl, % removal','Nitrogen, total, as NO3, [sludge]','Nitrogen, total, as NO3, [water]'],

                             'ARSENIC':['Arsenic','Arsenic total, dry weight, sludge','Arsenic, dissolved [as As]','Arsenic, dry weight','Arsenic, Inorganic','Arsenic, potentially dissolved','Arsenic, sludge tot. dry wt [as As]','Arsenic, total [as As]','Arsenic, total recoverable','Arsenic, total recoverable','Arsenic, trivalent dissolved','Arsenic, trivalent total','Arsenic, trivalent total recoverable'],

                             # TODO
                             'SEDIMENT':[],

                             'PHOSPHORUS, TOTAL':['Phosphorous 32, total','Phosphorous, in total orthophosphate','Phosphorous, total elemental','Phosphorous, total organic [as P]','Phosphorous, total recoverable','Phosphorous, total, in bottom deposits', 'Phosphorus [reactive as P]', 'Phosphorus Adsorption', 'Phosphorus Pentoxide','Phosphorus, brays','Phosphorus, dissolved','Phosphorus, dissolved reactive [drp as P]','Phosphorus, insoluble','Phosphorus, ortho','Phosphorus, sludge, total, dry weight [as P]','Phosphorus, Total','Phosphorus, total [as P]','Phosphorus, total [as P] [Can Production]','Phosphorus, total [as P] [credit]','Phosphorus, total [as P] [delivered]','Phosphorus, total [as P] [per season]','Phosphorus, Total [Avg Seasonal Load Cap]','Phosphorus, Total As P','Phosphorus, total percent removal','Phosphorus, total soluble [as PO4]'],

                             'NH3-N':['Ammonia nitrogen, total, [as N] 30 day', 'Nitrogen, ammonia [NH3-N]','Nitrogen, ammonia [NH3-N], [sludge]','Nitrogen, ammonia dissolved','Nitrogen, ammonia in bottom deposits','Nitrogen, ammonia per cfs of streamflow','Nitrogen, ammonia total [as N]','Nitrogen, ammonia total [as N] [per discharge]','Nitrogen, ammonia total [as N] [per season]','Nitrogen, ammonia total [as NH4]','Nitrogen, ammonia, percent removal','Nitrogen, ammonia, sludge solid','Nitrogen, ammonia, sludge, tot dry wgt','Nitrogen, ammonia, tot unionized [as N]','Nitrogen, ammonia, total [as NH3]','Nitrogen, AmmoniaTotal'],

                             'TOTAL SUSPENDED SOLIDS (TSS)':['Solids, total susp per production','Solids, total susp. per 1000 gallons','Solids, total susp. per batch','Solids, total susp. per cfs of streamflw','Solids, total suspd. non-volatile','Solids, Total Suspen [lbs/1000 lbs Product]','Solids, total suspended','Solids, total suspended [Can Production]','Solids, total suspended [per discharge]','Solids, total suspended [per season]','Solids, total suspended, loading rate','Solids, Total Suspended, Net Value'],

                             'TURBIDITY':['Turbidity','Turbidity [severity]','Turbidity, % increas over intake','Turbidity, background','Turbidity, change','Turbidity, hch turbidimiter','Turbidity, hellige [silicon dioxide]','Turbidity, increase over background','Turbidity, lab, NTU','Turbidity, sed stick','Turbidity, Test Failures','Turbidity, time above limit','Turbidity, transmissometer, %'],

                             'AMMONIA, TOTAL':['Ammonia [as N] + unionized ammonia','Ammonia & ammonium- total','Ammonia nitrogen, total, [as N] 30 day','Ammonia, unionized', 'Nitrogen, ammonia [NH3-N]','Nitrogen, ammonia [NH3-N], [sludge]','Nitrogen, ammonia dissolved','Nitrogen, ammonia in bottom deposits','Nitrogen, ammonia per cfs of streamflow','Nitrogen, ammonia total [as N]','Nitrogen, ammonia total [as N] [per discharge]','Nitrogen, ammonia total [as N] [per season]','Nitrogen, ammonia total [as NH4]','Nitrogen, ammonia, percent removal','Nitrogen, ammonia, sludge solid','Nitrogen, ammonia, sludge, tot dry wgt','Nitrogen, ammonia, tot unionized [as N]','Nitrogen, ammonia, total [as NH3]','Nitrogen, AmmoniaTotal'],

                             'THALLIUM':['Thallium 208','Thallium in bottom deposits [dry wgt]','Thallium, acid soluble','Thallium, calculated limit','Thallium, dissolved [as Tl]','Thallium, potentially dissolvd','Thallium, total [as Tl]','Thallium, total recoverable'],

                             'ZINC':['Zinc','Zinc in bottom deposits [dry wgt]','Zinc, dissolved [as Zn]','Zinc, percent removal','Zinc, potentially dissolved','Zinc, sludge solid','Zinc, sludge total','Zinc, sludge, total, dry weight [as Zn]','Zinc, total','Zinc, total [as Zn]','Zinc, total [as Zn] [Can Production]','Zinc, total [as Zn] [per discharge]','Zinc, total recoverable','Zinc, total recoverable','Zinc, total sludge']
                             }

# Convert the linkage dictionary into a dataframe so it can be easily merged with the TMDL dataset
pollutant_parameter_links = pd.DataFrame()
for pollutant in pollutant_parameter_link_dict.keys():
    pollutant_parameter_links = pd.concat([pollutant_parameter_links, pd.DataFrame({'pollutantName':[pollutant]*len(pollutant_parameter_link_dict[pollutant]), 'parameter_desc':pollutant_parameter_link_dict[pollutant]})])

In [72]:
# Merge with TMDLs dataset to produce a long-form DataFrame ready to merge with DMRs
TX_tmdls_merged = TX_tmdls_merged.merge(pollutant_parameter_links, on='pollutantName', how='inner')

In [73]:
# Merge with DMRs
TX_dmrs_tmdls_merged = TX_tmdls_merged.merge(TX_dmrs, on=['npdes_permit_id', 'parameter_desc'])

In [74]:
# Define incumbency
TX_dmrs_tmdls_merged['incumbent'] = (TX_dmrs_tmdls_merged['issue_date'] <= TX_dmrs_tmdls_merged['TMDLDate'])

In [75]:
summary_stats = TX_dmrs_tmdls_merged.groupby(['actionIdentifier', 'parameter_desc', 'value_type_code', 'incumbent'], as_index=False).agg(DMR_mean=pd.NamedAgg(column='dmr_value_standard_units', aggfunc=lambda x: x.mean()), DMR_mean_SE=pd.NamedAgg(column='dmr_value_standard_units', aggfunc=lambda x: x.sem()), n_permittees=pd.NamedAgg(column='npdes_permit_id', aggfunc=lambda x: x.nunique()))
summary_stats.head(100)

,actionIdentifier,parameter_desc,value_type_code,incumbent,DMR_mean,DMR_mean_SE,n_permittees
0,35935,Fecal coliform,Q2,False,5236.181818,2474.406745,26
1,35935,Fecal coliform,Q2,True,20841.636364,8243.668692,25
2,39028,E. coli,C2,False,8.539072,2.970578,27
3,39028,E. coli,C2,True,14.248705,6.902800,3
4,39028,E. coli,C3,False,61.206249,6.979893,26
5,39028,E. coli,C3,True,419.489209,113.035126,3
6,39030,E. coli,C2,False,25.643331,6.722257,26
7,39030,E. coli,C2,True,5.811185,1.220250,3
8,39030,E. coli,C3,False,69.166512,10.955951,26
9,39030,E. coli,C3,True,472.739855,348.438351,3


In [59]:
summary_stats_pivotted = pd.pivot(summary_stats, index=['actionIdentifier', 'parameter_desc', 'value_type_code'], columns='incumbent')

In [60]:
summary_stats_pivotted.columns = ['Newcomer DMR mean', 'Incumbent DMR mean', 'Newcomer DMR SE', 'Incumbent DMR SE', 'Newcomer n_permittees', 'Incumbent n_permittees']

In [61]:
summary_stats_pivotted['Welch test t-stat'] = (summary_stats_pivotted['Incumbent DMR mean']-summary_stats_pivotted['Newcomer DMR mean']) / np.sqrt((summary_stats_pivotted['Incumbent DMR SE'])**2 + (summary_stats_pivotted['Newcomer DMR SE'])**2)

In [62]:
summary_stats_pivotted

Newcomer DMR mean  \
actionIdentifier parameter_desc                      value_type_code                      
35935            Fecal coliform                      Q2                     5236.181818   
39028            E. coli                             C2                        8.539072   
                                                     C3                       61.206249   
39030            E. coli                             C2                       25.643331   
                                                     C3                       69.166512   
39035            E. coli                             C2                       11.498986   
                                                     C3                       29.230694   
                 E. coli, colony forming units [CFU] C2                        4.587500   
                                                     C3                        4.587500   
51281            E. coli                             C2                        3.463130   
                                                     C3                      125.377542   
52900            E. coli                             C2                        3.150130   
                                                     C3                      115.482841   
54900            E. coli                             C2                        1.285714   
                                                     C3                        9.238095   
54920            E. coli                             C2                        1.285714   
                                                     C3                        9.238095   
54921            E. coli                             C2                        1.285714   
                                                     C3                        9.238095   
54981            E. coli                             C2                        1.285714   
                                                     C3                        9.238095   
64680            E. coli                             C2                        3.316681   
                                                     C3                       47.025564   

                                                                      Incumbent DMR mean  \
actionIdentifier parameter_desc                      value_type_code                       
35935            Fecal coliform                      Q2                     20841.636364   
39028            E. coli                             C2                        14.248705   
                                                     C3                       419.489209   
39030            E. coli                             C2                         5.811185   
                                                     C3                       472.739855   
39035            E. coli                             C2                        24.823964   
                                                     C3                        99.049172   
                 E. coli, colony forming units [CFU] C2                              NaN   
                                                     C3                              NaN   
51281            E. coli                             C2                         5.360562   
                                                     C3                       245.189759   
52900            E. coli                             C2                        36.723605   
                                                     C3                       943.176091   
54900            E. coli                             C2                              NaN   
                                                     C3                              NaN   
54920            E. coli                             C2                              NaN   
                                                     C3                              NaN   
54921            E. coli                      